In [3]:
import folium
from folium.plugins import HeatMap
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic

In [4]:
wifi_count_data = pd.read_csv('count_wifi_loc.csv')
wifi_location = pd.read_csv('PublicWiFi.csv')
seoul_district_boundary = gpd.read_file('./district_seoul/LARD_ADM_SECT_SGG_11_202405.shp')

In [5]:
seoul_wifi_location = wifi_location[wifi_location['시도']=='서울특별시']
seoul_district_wifi = seoul_wifi_location.groupby('시군구')

In [ ]:
# 서울 중심 지도 생성
seoul_map = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

In [7]:
# # GeoJSON 데이터를 Folium에 추가
# folium.GeoJson(
#     seoul_district_boundary,  # GeoDataFrame 사용
#     name="Seoul Districts",
#     style_function=lambda x: {
#         'color': 'black',  # 경계선 색상
#         'weight': 2,  # 경계선 두께
#         'fillColor': 'gray',  # 채우기 색상
#         'fillOpacity': 0.1  # 채우기 투명도
#     }
# ).add_to(seoul_map)

In [8]:
# HeatMap 데이터 준비
heat_data = []
for index, entry in wifi_count_data.iterrows():
    lat, lon = entry['Latitude'], entry['Longitude']
    wifi_count = int(sum(entry.iloc[4:7]))  # 500m ~ 2km 열 합계를 Wi-Fi 개수로 사용
    heat_data.append([lat, lon, wifi_count])  # HeatMap용 데이터 준비

    # if wifi_count >= 750 :
    #     folium.Circle(
    #         location=[entry['Latitude'], entry['Longitude']],  # Wi-Fi 위치의 위도와 경도
    #         radius=2000,  # 원의 크기 (40m 반경)
    #         color='blue',  # 테두리 색상을 없앰
    #         fill=None,  # 원 내부 채우기
    #         fill_color='blue',  # 원 내부 색상
    #         fill_opacity=0.1  # 원의 투명도
    #     ).add_to(seoul_map)

    # Wi-Fi 개수 텍스트 추가
    folium.Marker(
        location=[lat, lon],
        icon=folium.DivIcon(
            html=f'''
                <div style="
                    font-size: 12px; 
                    font-weight: bold; 
                    color: black; 
                    padding: 2px; 
                    border-radius: 3px;">
                    {wifi_count}
                </div>
            '''
        )
    ).add_to(seoul_map)

# HeatMap 추가
HeatMap(heat_data).add_to(seoul_map)

In [9]:
# for _, row in seoul_wifi_location.iterrows():
#     folium.Circle(
#         location=[row['위도'], row['경도']],  # Wi-Fi 위치의 위도와 경도
#         radius=40,  # 원의 크기 (40m 반경)
#         color=None,  # 테두리 색상을 없앰
#         fill=True,  # 원 내부 채우기
#         fill_color='blue',  # 원 내부 색상
#         fill_opacity=0.2,  # 원의 투명도
#         popup=folium.Popup(row['ap명'], parse_html=True)  # Wi-Fi AP 이름 팝업
#     ).add_to(seoul_map)

In [10]:
# 결과 저장
seoul_map.save("seoul_wifi_heat.html")